In [4]:
import keras
from keras.datasets import mnist

from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Reshape, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.optimizers import Adam, SGD

from tensorflow.keras import backend as K
#from tensorflow.contrib.factorization.python.ops import clustering_ops

import tensorflow as tf

import sys 
import pandas as pd
import datetime as dt
import pickle
import os
import cv2 
import glob
import tqdm
import random
import matplotlib.pyplot as plt
import scipy.misc
#from scipy.misc import imresize
import numpy as np
import seaborn as sns

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
PACKAGE_PARENT = '../'
SCRIPT_DIR = os.path.dirname(os.path.realpath(os.path.join(os.getcwd(), os.path.expanduser('__file__'))))
sys.path.append(os.path.normpath(os.path.join(SCRIPT_DIR, PACKAGE_PARENT)))
from UTILS import cleaning_processing, HenDailyVariable_Origins, plot_scikit_lda, explained_var, sampen, kmeans_clustering,\
time_series_henColumn_tsRow, FB_daily
import config_origins as config

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
#initialise parameters
path_extracted_data = config.path_extracted_data
path_initial_data = config.path_initial_data
id_run = config.id_run
pal_ = config.pal_
dico_matching = config.dico_matching
dico_zone_order = config.dico_zone_order
dico_night_hour = config.dico_night_hour
dico_garden_opening_hour = config.dico_garden_opening_hour
path_extracted_data_visual_cons = os.path.join(path_extracted_data,'visual','IndividualConsistency')
#create a director if not existing
if not os.path.exists(path_extracted_data_visual_cons):
    os.makedirs(path_extracted_data_visual_cons)

path_extracted_data_visual_cons_img = os.path.join(path_extracted_data_visual_cons,'visual','MLPimages')
#create a director if not existing
if not os.path.exists(path_extracted_data_visual_cons_img):
    os.makedirs(path_extracted_data_visual_cons_img)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Download the data

In [7]:
#MLP
df = pd.read_csv(os.path.join(path_extracted_data, id_run+'_CLEANEDDATA.csv'), sep=';', parse_dates=['Timestamp', 'date']) 
print(df.shape)
df.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1186950, 16)


,HenID,PenID,system,Zone,model_prediction,Timestamp,date,next_record_date,duration,previous_record_date,previous_duration,next_zone,previous_zone,previous_previous_zone,correction_is_consecutive_equal_initial_zone,is_WG_open
0,hen_111,pen11,10 - 12,3_Zone,1.0,2020-09-30 00:01:25,2020-09-30,2020-09-30 00:01:38.000,0 days 00:00:13,NaN,NaN,2_Zone,NaN,NaN,False,False
1,hen_111,pen11,10 - 12,2_Zone,1.0,2020-09-30 00:01:38,2020-09-30,2020-09-30 00:14:38.000,0 days 00:13:00,2020-09-30 00:01:25.000,0 days 00:00:13,2_Zone,3_Zone,NaN,False,False
2,hen_147,pen8,8 - 9,5_Zone,1.0,2020-09-30 00:30:49,2020-09-30,2020-09-30 00:33:26.000,0 days 00:02:37,NaN,NaN,5_Zone,NaN,NaN,False,False


In [8]:
#bird info
df_FB_daily = FB_daily(config)
df_FB_daily['TagID'] = df_FB_daily['TagID'].map(lambda x: 'tag_'+str(x))
df_FB_daily['HenID'] = df_FB_daily['HenID'].map(lambda x: 'hen_'+str(x))
print(df_FB_daily.shape)
df_FB_daily.head(3)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


From the focalBirdinfo, you have 159 ative tags
(22905, 29)


,HenID,R-Pen,InitialStartDate,ShouldBeExcluded,StartDate,EndDate,FocalLegringID,Expected % of wrong class,Expected chance of wrong treatment,comments,...,TagID,EPI GENETIC TIMESTAMP,HA 23-11-2020,comportement,weight 23-11-2020,HA 04-01-2021,weight 04-01-2021,HA 01-02-21,weight 01-02-21,date
0,hen_1,1.0,10.06.2020,NaN,2020-09-29,2021-02-01,3,NaN,0,NaN,...,tag_105,9-date:2-2020,NaN,NaN,"1696,5",NaN,"1787,8",NaN,"1800,9",2020-09-30
1,hen_1,1.0,10.06.2020,NaN,2020-09-29,2021-02-01,3,NaN,0,NaN,...,tag_105,9-date:2-2020,NaN,NaN,"1696,5",NaN,"1787,8",NaN,"1800,9",2020-10-01
2,hen_1,1.0,10.06.2020,NaN,2020-09-29,2021-02-01,3,NaN,0,NaN,...,tag_105,9-date:2-2020,NaN,NaN,"1696,5",NaN,"1787,8",NaN,"1800,9",2020-10-02


In [9]:
df_available_date = df_FB_daily.groupby(['date'])['HenID'].agg(lambda x: list(x)).reset_index()
print(df_available_date.shape)
df_available_date.head(3)

(145, 2)


C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date,HenID
0,2020-09-30,"[hen_1, hen_2, hen_3, hen_4, hen_5, hen_6, hen..."
1,2020-10-01,"[hen_1, hen_2, hen_3, hen_4, hen_5, hen_6, hen..."
2,2020-10-02,"[hen_1, hen_2, hen_3, hen_4, hen_5, hen_6, hen..."


# Create MLP images

In [11]:
#sort by timestamp
df.sort_values(['Timestamp'], inplace=True)
#use up to the second level only
df['Timestamp'] = df['Timestamp'].map(lambda x: dt.datetime(x.year, x.month, x.day, x.hour, x.minute, x.second))
#remove the first record
df = df.drop_duplicates(subset=['HenID','Timestamp'], keep='last')
li_pen_done = ['pen12', 'pen11', 'pen8', 'pen4', 'pen5','pen10','pen3','pen9'] # so that we avoid coputing the time serie
li_pen = [v for v in list(df['PenID'].unique()) if v not in li_pen_done]
#li_pen.reverse()
#existing images
li_p = glob.glob(os.path.join(path_extracted_data_visual_cons_img, '*.png'))
li_p = [v.split('\\')[-1] for v in li_p]
for p in li_pen:
    print(p)
    try:
        df_ts = time_series_henColumn_tsRow(df[df['PenID']==p], config, col_ts='Zone', ts_with_all_hen_value=False, save=False, hen_time_series=False)
        li_hen = [v for v in df_ts.columns if v.startswith('hen')]
        li_date = df_ts['date'].unique()
        for d in tqdm.tqdm(li_date):
            li_available_hens = df_available_date[df_available_date['date']==d]['HenID'].values[0]
            for henID in li_hen:
                title = henID+'_'+str(d).split('T')[0]+'.png'
                if (title not in li_p) & (henID in li_available_hens):
                    li_zones_sec = df_ts[df_ts['date']==d][henID].tolist() 
                    #should be over the 24h, so the night hour should not change here
                    if len(li_zones_sec)==24*60*60:
                        M = np.zeros(shape=(max(dico_zone_order.values())+1, len(li_zones_sec))) #+1 car starts from 0
                        for zone_, order in dico_zone_order.items():
                            #avoid warning:
                            if zone_ in set(li_zones_sec):
                                M[order][list(np.where(np.array(li_zones_sec)==zone_)[0])] = 1
                        #plot and save
                        plt.clf() # clears the entire current figure instead of plt.figure() which will create a new one, and hence keeping all figures
                        ax = sns.heatmap(M, cbar=False, cmap=sns.light_palette("black", as_cmap=True))  
                        plt.xticks([])
                        plt.yticks([])
                        ax.invert_yaxis() # we want the zone 5 to be up just for our eyes :)
                        plt.savefig(os.path.join(path_extracted_data_visual_cons_img, title), format='png')
                        plt.close()
    except Exception as e:
        print('ERROR')
        print(e)
        print(p,d,henID)

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


pen3
in this time series there is 26 hens
The initial starting date in over all is: 2020-09-30 02:41:43, and the ending date will be: 2021-02-16 17:25:15
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-09-30 02:41:43, and the ending date will be: 2021-02-16 23:59:59
Total running time: 3.17 mn


100%|██████████████████████████████████████████████████████████████████████████████| 140/140 [2:45:21<00:00, 70.87s/it]


pen9
in this time series there is 26 hens
The initial starting date in over all is: 2020-09-30 02:58:38, and the ending date will be: 2021-02-16 17:27:50
But note that birds may have different ending and starting date which should be taken into account when computing variables
and after ending the last day at midnight : 2020-09-30 02:58:38, and the ending date will be: 2021-02-16 23:59:59


  0%|                                                                                          | 0/140 [00:00<?, ?it/s]

Total running time: 2.74 mn


100%|████████████████████████████████████████████████████████████████████████████████| 140/140 [45:40<00:00, 19.57s/it]


In [7]:
TODO

C:\Users\camil\anaconda3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


NameError: name 'TODO' is not defined

ERROR! Session/line number was not unique in database. History logging moved to new session 505


### data processing

In [ ]:
x_train = []
x_test = []
p = 0.1
li_image_train = random.sample(li_image, int(len(li_image)*(1-p)))
for path_ in tqdm.tqdm(li_image):
    image = cv2.imread(path_)
    b,g,r = cv2.split(image)           
    image = cv2.merge([r,g,b])
    if title=='FF':
    #resize to have less pixels
        image = imresize(image, size=(224,224,3))
        #remove surroundings
        image = image[7:-17,20:-36]
        #normalized
        image = image/255.
    if title=='representation_color_and_duration':
        #remove surroundings
        image = image[33:-20,:]         #remove simple the title
        image = imresize(image, size=(200,168,3))
        #normalized
        image = image/255.
    if path_ in li_image_train:
        x_train.append(image)
    else:
        x_test.append(image)

In [ ]:
#save
pickle.dump(li_image_train, 
            open(os.path.join(path_save_model_info,'_li_image_train.pkl'), 'wb'))
pickle.dump([x for x in li_image if x not in li_image_train], 
            open(os.path.join(path_save_model_info,'_li_image_test.pkl'), 'wb'))

In [ ]:
plt.imshow(image);

In [ ]:
x_test = np.array(x_test).reshape(len(x_test), len(x_test[0]), len(x_test[0][0]), len(x_test[0][0][0]))
x_test = x_test.astype('float32')
print(x_test.shape)

x_train = np.array(x_train).reshape(len(x_train), len(x_train[0]), len(x_train[0][0]), len(x_train[0][0][0]))
x_train = x_train.astype('float32')
print(x_train.shape)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1) #transform 2D 28x28 matrix to 3D (28x28x1) matrix
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255 #inputs have to be between [0, 1]
x_test /= 255

img_size = x_train.shape[1]
total_pixels = img_size * img_size
translator_factor = 2
translator_layer_size = int(total_pixels/translator_factor)
middle_factor = 2
middle_layer_size = int(translator_layer_size/middle_factor)

inputs = keras.Input(shape=(img_size,img_size,1), name='cat_image')
x = layers.Flatten(name = 'flattened_cat')(inputs) #turn image to vector.

x = layers.Dense(translator_layer_size, activation='relu', name='encoder')(x)
x = layers.Dense(middle_layer_size, activation='relu', name='middle_layer')(x)
x = layers.Dense(translator_layer_size, activation='relu', name='decoder')(x)


outputs = layers.Dense(total_pixels, activation='relu', name='reconstructed_cat')(x)
outputs = layers.Reshape((img_size,img_size,1))(outputs)

model = keras.Model(inputs=inputs, outputs=outputs)

model.summary()

In [ ]:
x_train.shape

In [ ]:
model = Sequential()
 
#1st convolution layer
model.add(Conv2D(16, (3, 3) #16 is number of filters and (3, 3) is the size of the filter.
, padding='same', input_shape=(x_train.shape[1],x_train.shape[2],x_train.shape[3])))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
 
#2nd convolution layer
model.add(Conv2D(2,(5, 5), padding='same')) # apply 2 filters sized of (3x3)
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4,4), padding='same'))

#3nd convolution layer
#model.add(Conv2D(2,(3, 3), padding='same')) # apply 2 filters sized of (3x3)
#model.add(Activation('relu'))
#model.add(MaxPooling2D(pool_size=(2,2), padding='same'))

#here compressed version
 
#4rd convolution layer
model.add(Conv2D(2,(5, 5), padding='same')) # apply 2 filters sized of (3x3)
model.add(Activation('relu'))
model.add(UpSampling2D((4, 4)))
 
#5rd convolution layer
model.add(Conv2D(16,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(UpSampling2D((2, 2)))

#6rd convolution layer
#model.add(Conv2D(16,(3, 3), padding='same'))
#model.add(Activation('relu'))
#model.add(UpSampling2D((2, 2)))

model.add(Conv2D(x_train.shape[3],(3, 3), padding='same'))
model.add(Activation('sigmoid'))

model.summary()

In [ ]:
id_ = 'test1_'+title
path_model = os.path.join(path_save_model_info,id_)
if not os.path.exists(path_model):
    os.makedirs(path_model)

In [ ]:
#if not enough improvment on the val loss, then we will reduce the learning rate, and after that if still no change then 
#early stopping 

#The checkpoint only includes the model weights
#save the network weights only when there is an improvement in classification accuracy on the validation dataset 
filepath = os.path.join(path_save_model_info,id_,'weights-improvement-{epoch:02d}-{val_loss:.2f}.h5')
#if save_best_only=True, the latest best model according to the quantity monitored will not be overwritten
checkpoint = keras.callbacks.ModelCheckpoint(filepath, 
                                             monitor='val_loss', 
                                             verbose=1, 
                                             save_best_only=False, 
                                             mode='max')

#to reduce learning rate (new_lr = lr*factor) when a metric has stopped improving
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                              patience=1, #patience before changing learning rate
                                              min_delta=0.0001,
                                              verbose=1,
                                              factor=0.1)

#Stop training when a monitored quantity has stopped improving. to avoid overfiting
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                                               min_delta=0.001, 
                                               patience=3,
                                               verbose=1, 
                                               mode='auto',
                                               baseline=None)

#create logs data for tensorboard usage
tensorboard = keras.callbacks.TensorBoard(log_dir=os.path.join(path_log,id_), 
                                          histogram_freq=0, #no histo
                                          write_graph=True, 
                                          write_grads=False,
                                          write_images=True)

callbacks_list = [reduce_lr, early_stopping, checkpoint, tensorboard] 
#then as usuall to load weights to an instantiate model: model.load_weights("weights.best.hdf5")

In [ ]:
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy') #'adadelta' Adam(lr=0.001)
model.fit(x_train, x_train, epochs=100, validation_data=(x_test, x_test), callbacks=callbacks_list)

In [ ]:
#save all the architecture, weights etc
model.save(os.path.join(path_save_model_info,id_,'model_all.h5')) 
#model = load_model(os.path.join(path_save_model_info,id_,'model_all.h5'))

In [ ]:
#Once TensorBoard is running, navigate your web browser to localhost:6006 to view the TensorBoard.
#tensorboard --logdir=D:\vm_exchange\AVIFORUM\data\extracted_info_mobility_VF\visual\FF_model_info\FF_model_logs

### restored images

In [ ]:
restored_imgs = model.predict(x_test)
for i in range(15):
    plt.imshow(x_test[i])
    plt.show()
    plt.imshow(restored_imgs[i])
    plt.show()
    print("----------------------------")

### define K-means

In [ ]:
unsupervised_model = tf.contrib.learn.KMeansClustering(
    7 #num of clusters
    , distance_metric = clustering_ops.SQUARED_EUCLIDEAN_DISTANCE
    , initial_clusters=tf.contrib.learn.KMeansClustering.RANDOM_INIT
)

### fit k-means

In [ ]:
int(model.layers[7].output.shape[1])

In [ ]:
print(x_test.shape)
x_test_pp = x_test[0:600].copy()
print(x_test_pp.shape)
print(model.layers[0].input)
print(model.layers[7].input)

In [ ]:
#layers = len(model.layers)
#for i in range(layers):
#    print(i, ". ", model.layers[i].output.get_shape())
#layer[7] is a compressed representation. It is size of (None, 7, 7, 2). this means 2 different 7x7 sized matrixes. We will 
#flatten these matrixes.
vo = model.layers[7].output.shape
get_3rd_layer_output = K.function([model.layers[0].input], [model.layers[7].output])
compressed = get_3rd_layer_output([x_test_pp])[0]
compressed = compressed.reshape(x_test_pp.shape[0],int(vo[1])*int(vo[2])*int(vo[3]))

def train_input_fn():
    data = tf.constant(compressed, tf.float32)
    return (data, None)

unsupervised_model.fit(input_fn=train_input_fn, steps=3000)

In [ ]:
#save all the architecture, weights etc
#unsupervised_model.save(os.path.join(path_save_model_info,id_,'unsupervised_model_all.h5')) 
#model = load_model(os.path.join(path_save_model_info,id_,'model_all.h5'))

### clustering the test sample

In [ ]:
clusters = unsupervised_model.predict(input_fn=train_input_fn)
import scipy.misc
id_img = 0
for k,i in tqdm.tqdm(enumerate(clusters)):
    cluster_id = i['cluster_idx']
    id_img = id_img+1
    features = x_test_pp[k]
    path_class_ = os.path.join(path_class, str(cluster_id))
    if not os.path.exists(path_class_):
        os.makedirs(path_class_)
    scipy.misc.imsave(os.path.join(path_class_,str(id_img)+'.png'), features)

### clustering all

for j in range(0, x_train.shape[0], 500):
    X = x_train[j-500,j]
    compressed = get_3rd_layer_output([X])[0]
    compressed = compressed.reshape(X.shape[0],int(vo[1])*int(vo[2])*int(vo[3]))

    def train_input_fn():
        data = tf.constant(compressed, tf.float32)
        return (data, None)

    clusters = unsupervised_model.predict(input_fn=train_input_fn)
    id_img = 0
    for k,i in tqdm.tqdm(enumerate(clusters)):
        cluster_id = i['cluster_idx']
        id_img = id_img+1
        features = X[k]
        path_class_ = os.path.join(path_class, title, str(cluster_id))
        if not os.path.exists(path_class_):
            os.makedirs(path_class_)
        scipy.misc.imsave(os.path.join(path_class_,str(id_img)+'.png'), features)